In [1]:
from llmcoder import LLMCoder

In [2]:
llmcoder = LLMCoder(
    # analyzers=["mypy_analyzer_v1", "signature_analyzer_v1"],
    analyzers=["signature_analyzer_v1"],
    max_iter=5,
    feedback_variant="coworker",
    n_procs=3,
    min_score=5,
    backtracking=False, 
    combining=True,
    verbose=True
    )

In [3]:
initial_code = '''
        total_generate_candidates = 0

        # Get the completions from OpenAI's API
        candidates = self.client.chat.completions.create(
            messages=conversation.messages,
            model=model,
            temperature=temperature,
            n=n)  # type: ignore

        total_generate_candidates += len(candidates.choices)

        # Count the number of tokens generated
        self.n_tokens_generated += sum([len(self.encoder.encode(message.message.content)) for message in candidates.choices])

        # Filter out completions that are repetitions of previous mistakes
        valid_choices = [completion for completion in candidates.choices if not self._is_bad_completion(completion.message.content)]
        print("[ME] The number of valid choices is: ", len(valid_choices))
        # Remove duplicates
        valid_unique_contents = list(set([choice.message.content for choice in valid_choices]))
        print("[ME] The number of valid unique contents is: ", len(valid_unique_contents))

        # If all completions are repetitions of previous mistakes, increase the temperature and the number of choices until we get a valid completion
        increased_temperature = temperature
        increased_n = n
        repetition = 0

        while len(valid_unique_contents) < n and repetition < max_retries:
            if self.verbose:
                print(f"[LLMcoder] Found {total_generate_candidates - len(valid_choices)} repeated mistakes, {len(valid_choices) - len(valid_unique_contents)} duplicates. Increasing temperature to {increased_temperature:.1f} and number of choices to {increased_n}... [repetition {repetition + 1}/{max_retries}]")

            # Sample new candidates
            candidates = self.client.chat.completions.create(
                messages=conversation.messages,
                model=model,
                temperature=increased_temperature,
                n=increased_n)  # type: ignore

            total_generate_candidates += len(candidates.choices)

            # Add the new completions to the list of valid completions
            # Filter out completions that are repetitions of previous mistakes
            valid_choices += [completion for completion in candidates.choices if not self._is_bad_completion(completion.message.content)]

            # Remove duplicates
            valid_unique_contents = list(set([choice.message.content for choice in valid_choices]))

            increased_temperature = min(max_temperature, increased_temperature + delta_temperature)
            increased_n = min(max_n, increased_n * factor_n)
            repetition += 1

        # If we still do not have valid choices, abort
        if len(valid_unique_contents) == 0 and repetition >= max_retries:
            if self.verbose:
                print("[LLMcoder] Coul
'''

In [4]:
llmcoder.complete_with_feedback(code=initial_code, min_score=5, n=3)

[LLMcoder] Creating first completions...
[LLMcoder] Choosing conversation R with score 0
[LLMCoder] The number of valid choices is:  3
[LLMCoder] The number of valid unique contents is:  3
[LLMcoder] Analyzing 3 completions...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[Signatures] No problematic functions or classes found in the context.
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[Signatures] No problematic functions or classes found in the context.
[Signatures] No problematic functions or classes found in the context.
[LLMcoder] Have 3 conversations:
[LLMcoder] Passing   Score     Prob      Path
[LLMcoder] True      0         0.3333    ['R', 0]
[LLMcoder] True      0         0.3333    ['R', 1]
[LLMcoder] True      0         0.3333    ['R', 2]
[LLMcoder] Starting feedback loop...
[LLMcoder] Starting feed

'd not find any valid choice after", repetition, "retries. Aborting...")\n            return []'

In [5]:
messages = [
    {
        "role": "system",
        "content": llmcoder.system_prompt
    }, {
        "role": "user",
        "content": initial_code
    }
]

In [6]:
from openai import OpenAI
from llmcoder.utils import get_openai_key

client = OpenAI(api_key = get_openai_key())
completions = client.chat.completions.create(messages=messages, model="gpt-3.5-turbo", temperature=0.7, n = 3)

In [7]:
conversation = llmcoder.conversations.pop(keep=True)

In [8]:
conversation.add_message({'role': 'user', 'content': initial_code})
conversation.add_message({'role': 'assistant', 'content': completions.choices[0].message.content})

In [10]:
conversation2 = conversation.copy().add_message({'role': 'assistant', 'content': completions.choices[1].message.content})
llmcoder.conversations.push(conversation2)

In [11]:
for i, c in enumerate([conversation, conversation2]):
    print(f'[LLMCoder] Conversation {i} has messages: {c.messages}')

[LLMCoder] Conversation 0 has messages: [{'role': 'system', 'content': "You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.\nYou are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.\nPrecisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.\nYour code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.\nYour unique capability is to provide completions without altering, repeating, or commenting on the original code.\nYou offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.\nThis approach makes you an ideal assistant for users seeking straightforward and efficient 

In [12]:
if len(conversation.analyses) > 0 and len(conversation2.analyses) > 0:
    combining_prompt = llmcoder._combining_prompt_template(conversation, conversation2,
                                                                # If there is feedback available from previous analyses, add it to the prompt
                                                                [str(result['message']) for result in conversation.analyses[-1].values()
                                                                    if (not result['pass'] or result['type'] == "info")],
                                                                [str(result['message']) for result in conversation2.analyses[-1].values()
                                                                    if (not result['pass'] or result['type'] == "info")])
elif len(conversation.analyses) > 0:
    combining_prompt = llmcoder._combining_prompt_template(conversation, conversation2,
                                                        [str(result['message']) for result in conversation.analyses[-1].values()
                                                            if (not result['pass'] or result['type'] == "info")],
                                                        None)
elif len(conversation2.analyses) > 0:
    combining_prompt = llmcoder._combining_prompt_template(conversation, conversation2,
                                                        None,
                                                        [str(result['message']) for result in conversation2.analyses[-1].values()
                                                            if (not result['pass'] or result['type'] == "info")])
else:
    combining_prompt = llmcoder._combining_prompt_template(conversation, conversation2, None, None)
    

In [14]:
conversation.add_message({'role': 'user', 'content': combining_prompt})
llmcoder.conversations.push(conversation)
llmcoder._get_completions_for(conversation, llmcoder.model_feedback, temperature=0.7, n=3)

[LLMCoder] The number of valid choices is:  3
[LLMCoder] The number of valid unique contents is:  3
WE HAVE VALID CHOICES!!!!!!!!
[LLMcoder] Analyzing 3 completions...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[Signatures] No problematic functions or classes found in the context.
[LLMcoder] Analyzing code in coworker mode...
[LLMcoder] Running signature_analyzer_v1...
[Signatures] No problematic functions or classes found in the context.
[Signatures] No problematic functions or classes found in the context.
